# POLI 175 - Lecture 13

## Beyond Linearity (cont'd) and Tree-based models

## Class Examples

In [1]:
## Packages Here
using DataFrames
using MLJ, MLJIteration
import MLJLinearModels, MLJBase, MLJModels
import MultivariateStats, MLJMultivariateStatsInterface
import CSV, Plots, GLM, StatsBase, Random
import LaTeXStrings, StatsPlots, Lowess, Gadfly, RegressionTables
import CovarianceMatrices, Econometrics, LinearAlgebra, MixedModelsExtras
import Missings, StatsAPI, FreqTables, EvalMetrics
import NearestNeighborModels

# Adapted from @xiaodaigh: https://github.com/xiaodaigh/DataConvenience.jl
function onehot!(df::AbstractDataFrame, 
        col, cate = sort(unique(df[!, col])); 
        outnames = Symbol.(col, :_, cate))
    transform!(df, @. col => ByRow(isequal(cate)) .=> outnames)
end

onehot! (generic function with 2 methods)

## Class Examples

In [2]:
## Loading the data
chile = CSV.read(
    download("https://raw.githubusercontent.com/umbertomig/POLI175julia/main/data/chilesurvey.csv"), 
    DataFrame,
    missingstring = ["NA"]
); dropmissing!(chile)
chile.voteyes = ifelse.(chile.vote .== "Y", 1, 0)

# One-hot encoding (we will learn a better way to do it later)
onehot!(chile, :region);
onehot!(chile, :education);
onehot!(chile, :sex);

# Drop reference categories
select!(chile, Not(:region, :income, :population, :sex, :education, :region_C, :education_P, :sex_M))

# Checking
first(chile, 3)

Row,age,statusquo,vote,voteyes,region_M,region_N,region_S,region_SA,education_PS,education_S,sex_F
,Int64,Float64,String1,Int64,Bool,Bool,Bool,Bool,Bool,Bool,Bool
1,65,1.0082,Y,1,false,true,false,false,false,false,false
2,29,-1.29617,N,0,false,true,false,false,true,false,false
3,38,1.23072,Y,1,false,true,false,false,false,false,true


## Class Examples

In [3]:
## Education Expenditure Dataset
educ = CSV.read(download("https://raw.githubusercontent.com/umbertomig/POLI175julia/main/data/educexp.csv"), DataFrame)

# Processing
educ.educ_log = log.(educ.education);
educ.income_log = log.(educ.income)
educ.urban_log = log.(educ.urban)
educ.young_log = log.(educ.young)

# Checking
first(educ, 3)

Row,education,income,young,urban,states,educ_log,income_log,urban_log,young_log
,Int64,Int64,Float64,Int64,String3,Float64,Float64,Float64,Float64
1,189,2824,350.7,508,ME,5.24175,7.94591,6.23048,5.85993
2,169,3259,345.9,564,NH,5.1299,8.08918,6.33505,5.84615
3,230,3072,348.5,322,VT,5.43808,8.03008,5.77455,5.85364


## Beyond Linearity

## Basis Functions

Family of transformations $b_i(X)$ such that, instead of fitting a regression model in $X$, we fit:

$$ y_i \ = \ \beta_0 + \beta_1b_1(x_i) + \beta_2b_2(x_i) + \cdots + \beta_kb_k(x_i) + \varepsilon_i $$

In this sense:
- Polynomial Regression: $b_j(x_i) = x_i^j$
- Constant-piecewise Regression: $b_j(x_i) = C_j(x_i) = I(c_j \leq X < c_{j+1})$
    
Meaning: they are cases of a broader set of basis functions.

## Polynomial Regression

Expands the default model ($y_i = \beta_0 + \beta_1x_i + \varepsilon_i$) to consider a polynomial $d$:

$$ y_i = \beta_0 + \beta_1x_i + \beta_2x_i^2 +\cdots + \beta_dx_i^d + \varepsilon_i $$
                     
Lots of flexibility here, but we seldom use $d>4$ because then it gets *excessively* flexible.

Prediction very straightforward:

$$ \hat{f}(x_0) = \hat{\beta}_0 + \hat{\beta}_1x_0 + \hat{\beta}_2x_0^2 +\cdots + \hat{\beta}_dx_0^d $$

## Polynomial Regression

![img](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/polyreg1.png?raw=true)

## Polynomial Regression

Let us try to explain `education` expenditure using per capita `income`.

In [1]:
# Checking
first(educ, 3)

NameError: name 'first' is not defined

## Polynomial Regression

Let us try to explain `education` expenditure using per capita `income`.

In [ ]:
educaux = educ[:, ["education", "income"]];
educaux.income2 = educaux.income .^ 2;
educaux.income3 = educaux.income .^ 3;
educaux.income4 = educaux.income .^ 4;
educaux.income5 = educaux.income .^ 5;
educaux.income6 = educaux.income .^ 6;

In [ ]:
y, X = unpack(
    educaux, ## Variables we need
    ==(:education);                        ## Target (all else features...)
    :education   => Continuous,            ## Var types
    :income      => Continuous,
    :income2     => Continuous,
    :income3     => Continuous,
    :income4     => Continuous,
    :income5     => Continuous,
    :income6     => Continuous,
);

## Polynomial Regression

In [ ]:
linreg = MLJLinearModels.LinearRegressor()

In [ ]:
# 3-Fold CV
cv3 = CV(
    nfolds = 3,  ## Number of cross-validating folds
    rng = 45321  ## Random seed
);

## Polynomial Regression

In [ ]:
## Evaluate the Model
vars = Vector{String}()
for i in ("income", "income2", "income3", "income4", "income5", "income6")
    push!(vars, i)
    println(
        evaluate(linreg,                   ## Model
            X[:, vars],                    ## Training features set
            y,                             ## Training target set
            resampling = cv3,              ## Resampling strategy (in this case, 3-Fold CV)
            measure = [l2, rmse],          ## Measures
            verbosity = 0)
    )
end

I view the linear regression as the best for this relationship.

# Tree-based methods

## Tree-based methods

Tree-based methods consist of segmenting the predictors' space into many regions

Then, use these regions to predict the target variable.
- We use heuristics here, such as the variable's mean in the region.
    
This approach is called the `decision tree method`

By itself it is terrible. But there are methods that improve efficiency considerably.
- At the cost of interpretability

## Decision Trees

May be applied to regression and classification.

Example: Suppose you want to predict salary of baseball players using the numbers of years playing and the hits:

A regression tree would perform the following predictions:

![img](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/tree1.png?raw=true)

## Decision Trees

This is how the fitted parameters segment the data:

![tree2](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/tree2.png?raw=true)

## Decision Trees

Formally:

1. Regression trees divide the predictors' space into distinct and non-overlapping regions $R_1$, $R_2$, $\cdots$, $R_J$.

2. For all observations within $R_j$, we make the same prediction.
    
How to construct the $R_1, R_2, \cdots, R_J$ space?

## Decision Trees

One theoretical solution would be to fit a regression tree that search for regions that minimize the Residual Sum of Squares:

$$ \sum_{j=1}^J\sum_{i \in R_j} (y_i - \hat{y}_{R_j})^2 $$

But it is easy to see that solving this is computationally infeasible.

- Multiple variables would amount to multiple regions.

- Many combinations and regions would be possible.

## Decision Trees

The way we actually fit this is called the `greedy` approach:

1. We consider one variable at a time.

2. We split it recursively.

3. We consider the regions that divide the space into two half-spaces:

$$ R_1(j, s) = \{X | X_j < s\} \quad and \quad R_2(j, s) = \{X | X_j \geq s\} $$

## Decision Trees

Minimizing:

$$ \sum_{i: \ x_i \in R_1(j,s)} (y_i - \hat{y}_{R_1})^2 + \sum_{i: \ x_i \in R_2(j,s)} (y_i - \hat{y}_{R_2})^2$$

With $\hat{y}_{R_j}$ the mean of $y$ in the $j$-th half-space.

## Decision Trees

This could be the optimal, but never the greedy approach:

![tree3](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/tree3.png?raw=true)

## Decision Trees

This represents the greedy approach:

![tree4](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/tree4.png?raw=true)

## Decision Trees

And this is the levels / the regression tree:

![tree5](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/tree5.png?raw=true)

## Decision Trees

This approach will likely overfit the data. We need them to `prune` our tree a bit.

This involves removing parts of our tree that are not being helpful in the testing set predictions.

One heuristic is to use the size of the tree to penalize it. This is called `cost complexity pruning`

For all $T \subset T_0$:

$$ \sum_{m=1}^{|T|} \sum_{i: \ x_i \in R_m} (y_i - \hat{y}_{R_m})^2 + \alpha |T| $$

## Decision Trees

**Regression Tree Algorithm:**

1. Use recursive binary split (greedy approach) to grow a large tree on the training data. Stop when the terminal node reaches fewer than some threshold minimum number of observations.

2. Apply the cost-complexity pruning to the large tree to obtain a sequence of trees $T_1 \subset T_2 \subset \cdots \subset T_0$, as a function of $\alpha$.

## Decision Trees

**Regression Tree Algorithm:**

3. Use K-fold cross-validation to choose $\alpha$, i.e., for each $k \in \{1, 2, \cdots, K\}$:
    - Repeat steps 1 and 2 on all but $k$th fold of training data
    - Compute the MSE in the left-out fold
    - Pick $\alpha$ that minimizes the MSE.

4. Fit the corresponding tree for the optimal $\alpha$.

## Decision Trees

![tree6](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/tree6.png?raw=true)

## Classification Trees

Very similar approach, but you change the prediction and the statistic we look at.

1. We predict that all classes belong to the `most commonly occurring class` in the data.

2. We look at the `classification error rates` to grow our trees.

## Classification Trees

Let $\hat{p}_{mk}$ the proportion of cases in the $m$-th region that belong to the $k$-th class.

The error rate is:

$$ E = 1 - \max_k (\hat{p}_{mk}) $$

## Classification Trees

But, the classification error is sensitive to the size of the tree. Therefore, it is preferable to also look at the following:

1. The `Gini index`:

$$ G = \sum_k \hat{p}_{mk}(1 - \hat{p}_{mk}) $$

2. Or the `Entropy` level:
    
$$ D = - \sum_k \max_k \hat{p}_{mk}\log(\hat{p}_{mk}) $$

## Classification Trees

[Gini index](https://en.wikipedia.org/wiki/Gini_coefficient): measure of *node purity*: 

- Small values indicate that all $\hat{p}_{mk}$ are either close to zero or one.

[Entropy](https://en.wikipedia.org/wiki/Entropy):

- Easy to see that: $-\hat{p}_{mk}\log(\hat{p}_{mk}) \geq 0$

- It will take values close to zero if $\hat{p}_{mk}$ is close to either zero or one.

## Classification Trees

Feel free to use the measurement that you prefer.

But, minimizing error rates could be preferable when classification accuracy is the target.

In any case, you should cross-validate your analysis to pick the best parameters/tree size.

## Classification Trees

Classification in the book's heart disease data (before pruning):

![tree7](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/tree7.png?raw=true)

## Classification Trees

Classification in the book's heart disease data (after pruning):

![tree8](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/tree8.png?raw=true)

## Decision Trees x Linear Models

Linear regression:

$$ f(X) \ = \ \beta_0 + \sum_{j=1}^p X_j\beta_j $$

Decision Trees:

$$ f(X) \ = \ \sum_{m=1}^M c_m \cdot \mathbb{1}(X \in R_m) $$

Which model is better? No easy answer.

## Decision Trees x Linear Models

Regression is better:

![tree9](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/tree9.png?raw=true)

## Decision Trees x Linear Models

Decision tree is better:

![tree10](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/tree10.png?raw=true)

## Decision Trees

**Pros**:

1. Easy to explain.

2. Some argue that it mirrors human decision-making.

3. Allow for graphical display (kind of pretty, if you ask me...).

4. Easily handle qualitative predictors.

## Decision Trees

**Cons**:

1. Do not have the same level of accuracy as some predictive regression models.
    
2. Can be *non-robust*: small changes in data make up to significant changes in final estimation.

# Questions?

# See you next class
